# Projet 2

### Importation des modules usuels

In [2]:
import numpy as np
import pandas as pd

###  Chargement et traitement des données

In [3]:
geo = pd.read_csv("correspondance-code-insee-code-postal.csv", sep=';', usecols=range(11), index_col="Code INSEE")


geo = geo.sort_index()
geo[['Latitude', 'Longitude']] = geo["geo_point_2d"].str.extract("(.*), (.*)").astype(float)
geo.head()


,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude
Code INSEE,,,,,,,,,,,,
01001,01400,L'ABERGEMENT-CLEMENCIAT,AIN,RHONE-ALPES,Commune simple,242.0,1565.0,0.8,"46.1534255214, 4.92611354223","{""type"": ""Polygon"", ""coordinates"": [[[4.926273...",46.153426,4.926114
01002,01640,L'ABERGEMENT-DE-VAREY,AIN,RHONE-ALPES,Commune simple,483.0,912.0,0.2,"46.0091878776, 5.42801696363","{""type"": ""Polygon"", ""coordinates"": [[[5.430089...",46.009188,5.428017
01004,01500,AMBERIEU-EN-BUGEY,AIN,RHONE-ALPES,Chef-lieu canton,379.0,2448.0,13.4,"45.9608475114, 5.3729257777","{""type"": ""Polygon"", ""coordinates"": [[[5.386190...",45.960848,5.372926
01005,01330,AMBERIEUX-EN-DOMBES,AIN,RHONE-ALPES,Commune simple,290.0,1605.0,1.6,"45.9961799872, 4.91227250796","{""type"": ""Polygon"", ""coordinates"": [[[4.895580...",45.996180,4.912273
01006,01300,AMBLEON,AIN,RHONE-ALPES,Commune simple,589.0,602.0,0.1,"45.7494989044, 5.59432017366","{""type"": ""Polygon"", ""coordinates"": [[[5.614854...",45.749499,5.594320


## Projet maison n° 1

* Ajoutez une colonne 'CP Ville' avec le Code postal + un espace + et le nom de la Ville.
* Ecrivez une fonction qui détermine la commune la plus proche d'un point à partir de sa latitude et sa longitude.

* Ajoutez une fonction de conversion pour pouvoir utiliser la première fonction avec un GPS (degrés, minutes, secondes).

In [4]:
# Ajoutez une colonne 'CP Ville' avec le Code postal + un espace + et le nom de la Ville.
geo['CP Ville'] = geo['Code Postal'] + " " + geo['Commune']
geo.head(2)

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude,CP Ville
Code INSEE,,,,,,,,,,,,,
01001,01400,L'ABERGEMENT-CLEMENCIAT,AIN,RHONE-ALPES,Commune simple,242.0,1565.0,0.8,"46.1534255214, 4.92611354223","{""type"": ""Polygon"", ""coordinates"": [[[4.926273...",46.153426,4.926114,01400 L'ABERGEMENT-CLEMENCIAT
01002,01640,L'ABERGEMENT-DE-VAREY,AIN,RHONE-ALPES,Commune simple,483.0,912.0,0.2,"46.0091878776, 5.42801696363","{""type"": ""Polygon"", ""coordinates"": [[[5.430089...",46.009188,5.428017,01640 L'ABERGEMENT-DE-VAREY


In [5]:
# fonction recherche de ville
def ville(lat, long):
    # Calcul de distance entre deux points
    ptlat = geo['Latitude'].apply(lambda x : (x-lat)**2)  # On crée une liste de distance pour la latitude
    ptlong = geo['Longitude'].apply(lambda y : (y-long)**2)  # On crée une liste de distance pour la longitude
    distance = np.sqrt(ptlat + ptlong) # pythagore pour calculer toutes les distances
    return geo.loc[distance.idxmin(), 'Commune']   # On récup la ville (dans la liste) avec la distance min 
        # .loc : Access a group of rows and columns by label(s) or a boolean array.
        # .idxmin() : Return index of first occurrence of minimum over requested axis.

    pass


In [6]:
# conversion degrés, minutes, secondes => décimal
def dms2dec(deg, mn, sec):
    return deg + mn / 60 + sec / 3600

In [7]:
# on applique la fonction à une coordonnée tirée au hasard
# init du random
np.random.seed(0)

a, b = 41.5, 51.1  # latitude min et max de la France métropolitaine
lat = (b - a) * np.random.random() + a
a, b = -5.1, 9.5  # longitude min et max de la France métropolitaine
long = (b - a) * np.random.random() + a

ville(lat, long)

'BOSJEAN'

In [8]:
# à partir de coordonnées GPS précises
ville(dms2dec(48, 42, 52), dms2dec(2, 14, 45))

'PALAISEAU'

## Projet maison n°2

La colonne "geo_shape" comporte des chaines de catactères au format JSON. Elles représentent les formes géométriques des communes qui sont soit des polygones soit composées de plusieurs polygones.

* Utiliser la librairie Python json pour parser la colonne "geo_shape".
* Donner le décompte des valeurs accédées avec la clé "type".
* Donner le décompte des longueurs des listes accédées avec la clé "coordinates".
* Quelle commune est la plus complexe géométriquement ?
* Quelle commune est la seconde la plus complexe géométriquement ?
* Quelles sont les villes qui sont de type "Polygon" mais dont la longueur des listes accédées avec la clé "coordinates" vaut 2 ?
* Pour ces villes vérifier que le premier polygone contient bien le second (enclave). NB : installer la librairie shapely, utiliser la classe Polygon de shapely.geometry. Sur Windows shapely peut nécessiter d'installer la dll "geos_c.dll" dans le répertoire "Library/bin" de votre environnement Python.

In [9]:
# Utiliser la librairie Python json pour parser la colonne "geo_shape".

import json
geo_shape = geo['geo_shape'].apply(lambda x : json.loads(x))
geo['geo_shape_type'] = geo_shape.apply(lambda x : x['type'])
geo['geo_shape_coordinates'] = geo_shape.apply(lambda x : x['coordinates'])


In [10]:
geo.head()

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude,CP Ville,geo_shape_type,geo_shape_coordinates
Code INSEE,,,,,,,,,,,,,,,
01001,01400,L'ABERGEMENT-CLEMENCIAT,AIN,RHONE-ALPES,Commune simple,242.0,1565.0,0.8,"46.1534255214, 4.92611354223","{""type"": ""Polygon"", ""coordinates"": [[[4.926273...",46.153426,4.926114,01400 L'ABERGEMENT-CLEMENCIAT,Polygon,"[[[4.92627342692396, 46.1200517234555], [4.921..."
01002,01640,L'ABERGEMENT-DE-VAREY,AIN,RHONE-ALPES,Commune simple,483.0,912.0,0.2,"46.0091878776, 5.42801696363","{""type"": ""Polygon"", ""coordinates"": [[[5.430089...",46.009188,5.428017,01640 L'ABERGEMENT-DE-VAREY,Polygon,"[[[5.430089291292173, 45.98275997279625], [5.4..."
01004,01500,AMBERIEU-EN-BUGEY,AIN,RHONE-ALPES,Chef-lieu canton,379.0,2448.0,13.4,"45.9608475114, 5.3729257777","{""type"": ""Polygon"", ""coordinates"": [[[5.386190...",45.960848,5.372926,01500 AMBERIEU-EN-BUGEY,Polygon,"[[[5.386190824025674, 45.930927620632], [5.374..."
01005,01330,AMBERIEUX-EN-DOMBES,AIN,RHONE-ALPES,Commune simple,290.0,1605.0,1.6,"45.9961799872, 4.91227250796","{""type"": ""Polygon"", ""coordinates"": [[[4.895580...",45.996180,4.912273,01330 AMBERIEUX-EN-DOMBES,Polygon,"[[[4.895580369408855, 45.97991618124606], [4.8..."
01006,01300,AMBLEON,AIN,RHONE-ALPES,Commune simple,589.0,602.0,0.1,"45.7494989044, 5.59432017366","{""type"": ""Polygon"", ""coordinates"": [[[5.614854...",45.749499,5.594320,01300 AMBLEON,Polygon,"[[[5.614854625053272, 45.734638855121595], [5...."


In [121]:
# Donner le décompte des valeurs accédées avec la clé "type".
geo['geo_shape_type'].value_counts()

Polygon         36670
MultiPolygon       72
Name: geo_shape_type, dtype: int64

In [122]:
# Donner le décompte des longueurs des listes accédées avec la clé "coordinates".

longlist = geo['geo_shape_coordinates'].apply(len)
longlist.value_counts()

1    36660
2       80
4        1
3        1
Name: geo_shape_coordinates, dtype: int64

In [135]:
# Quelle commune est la plus complexe géométriquement ?

geo.loc[geo['geo_shape_coordinates'].apply(len) == 4, 'Commune']

Code INSEE
83069    HYERES
Name: Commune, dtype: object

In [137]:
# Quelle commune est la seconde la plus complexe géométriquement ?

geo.loc[geo['geo_shape_coordinates'].apply(len) == 3, 'Commune']

Code INSEE
27701    VAL-DE-REUIL
Name: Commune, dtype: object

In [141]:
# Quelles sont les villes qui sont de type "Polygon" 
# mais dont la longueur des listes accédées avec la clé "coordinates" vaut 2 ?

polygone2 = geo.loc[(geo['geo_shape_type']=='Polygon') & (geo['geo_shape_coordinates'].apply(len) == 2)]
polygone2['Commune']


Code INSEE
02232               COYOLLES
05052               EYGLIERS
09042    LA BASTIDE-DE-SEROU
22209              PLOUBALAY
2A272                SARTENE
2B049              CALENZANA
48080               LANGOGNE
67486              SUNDHOUSE
68078              EGUISHEIM
71028            BEAUVERNOIS
Name: Commune, dtype: object

In [147]:
# Pour ces villes vérifier que le premier polygone contient bien le second (enclave). 
# NB : installer la librairie shapely, utiliser la classe Polygon de shapely.geometry. 
# Sur Windows shapely peut nécessiter d'installer la dll "geos_c.dll" dans le répertoire 
# "Library/bin" de votre environnement Python.

from shapely.geometry import Polygon
villePoly2 = polygone2.copy() 

villePoly2['Enclave'] = villePoly2['geo_shape_coordinates'].apply(lambda x : Polygon(x[0]).contains(Polygon(x[1])))
villePoly2[['Commune', 'geo_shape_coordinates', 'Enclave']]

,Commune,geo_shape_coordinates,Enclave
Code INSEE,,,
02232,COYOLLES,"[[[3.037897560658703, 49.16921270775551], [3.0...",True
05052,EYGLIERS,"[[[6.719912606601575, 44.70392733592357], [6.7...",True
09042,LA BASTIDE-DE-SEROU,"[[[1.423960819701936, 42.99550854650971], [1.4...",True
22209,PLOUBALAY,"[[[-2.11237709725211, 48.52896113651634], [-2....",True
2A272,SARTENE,"[[[8.940487655060174, 41.490088793265336], [8....",True
2B049,CALENZANA,"[[[8.911220447280732, 42.41019895400797], [8.8...",True
48080,LANGOGNE,"[[[3.850531831651621, 44.68745054452225], [3.8...",True
67486,SUNDHOUSE,"[[[7.594939251325704, 48.23228567615893], [7.5...",True
68078,EGUISHEIM,"[[[7.281156117554148, 48.02598189605906], [7.2...",True
